# Jupyter Notebook UI to analyze baseline data from tap-habituation experiments!

### Beginner Essentials:
1. Shift-Enter to run each cell. After you run, you should see an output "done step #". If not, an error has occured
2. When inputting your own code/revising the code, make sure you close all your quotation marks '' and brackets (), [], {}.
3. Don't leave any commas (,) hanging! (make sure an object always follows a comma. If there is nothing after a comma, remove the comma!
4. Learning to code? Each line of code is annotated to help you understand how this code works!

**Run all cells/steps sequentially, even the ones that do not need input**

## Step-by-Step Analysis of the Jupyter Notebook

| Step | Purpose | Key Actions |
|------|---------|-------------|
| **1. Import Packages** | Load required Python libraries for data analysis | Imports `pandas`, `numpy`, `matplotlib`, etc. | 
| **2. Pick Filepath** | Select the folder containing experimental data files (.dat or .trv) | Input required: Uses `FileChooser` widget to select directory | 
| **3. User-Defined Variables** | Set experiment parameters | Defines: `bin`  | 
| **4. Construct Filelist** | Find all files in selected folder | Sets working directory and scans `folder_path` using; Displays no. of `.trv` files found in the folder |
| **5. Process Data Function** | Define functions to load, clean, and analyze raw data | - `ProcessData()`: Loads files, calculates metrics (reversal probability, speed) |
| **6.1 Process Data** | Apply processing to all strains| - Checks `filelist` for unique strain names (e.g., "N2") <br>- Runs `ProcessData()` for each strain | 
| **7. Grouping & Naming** | Combine data from all strains | - Concatenates DataFrames<br>- Assigns dataset names (e.g., "N2") | 
| **Output CSV** | Save processed data | Exports `Baseline_data` to CSV |

### Key Notes:
- User Input Required: Steps 2 (file selection), 3 (parameters), 6.1 (strain verification)
- Output: Final CSV contains all analyzed tap response data

# 1. Importing Packages Required (No input required, just run)

In [1]:
import pandas as pd #<- package used to import and organize data
import numpy as np #<- package used to import and organize data
import seaborn as sns #<- package used to plot graphs
from matplotlib import pyplot as plt #<- package used to plot graphs
import os #<- package used to work with system filepaths
from ipywidgets import widgets #<- widget tool to generate button
from IPython.display import display #<- displays button
from ipyfilechooser import FileChooser
# from tkinter import Tk, filedialog #<- Tkinter is a GUI package
from tqdm.notebook import tqdm
# import dask.dataframe as dd
# import pingouin as pg
pd.set_option('display.max_columns', 50)
print("done step 1")

done step 1


## 2. Pick filepath (just run and click button from output)

Run the following cell and click the button 'Select Folder' to pick a filepath.

**Important: Later on, this script uses the total file path for each file to import and group data. That means if your folder has whatever your strain is named, the script will not work.**

(ex. if your folder has "N2" in it this script sees all files inside this folder as having the "N2" search key)

**An easy fix is to just rename your folder to something else (make your strains lower-case, or just have the date)**

In [8]:
starting_directory = '/Volumes'
chooser = FileChooser(starting_directory)
display(chooser)

FileChooser(path='/Volumes', filename='', title='', show_hidden=False, select_desc='Select', change_desc='Chan…

In [10]:
print(chooser.selected_path)
folder_path=chooser.selected_path

/Volumes/RankinLabMehak_SSD/Glia_Genes_Screen_2025


In [11]:
screens = ['PD_Screen', 'ASD_Screen', 'G-Proteins_Screen', 'Glia_Genes_Screen', 'Neuron_Genes_Screen', 'Miscellaneous', 'ASD_WGS_Screen']

screen_chooser = widgets.Select(options=screens, value=screens[0], description='Screen:')
display(screen_chooser)

Select(description='Screen:', options=('PD_Screen', 'ASD_Screen', 'G-Proteins_Screen', 'Glia_Genes_Screen', 'N…

In [13]:
Screen=screen_chooser.value
print(Screen)

Glia_Genes_Screen


# 3. User Defined Variables (Add input here)

Here, we add some constants to help you blaze through this code.

3.1: Setting time bins


3.2: Setting view range for your graph
- Top, bottom = y axis view range
- left, right = x axis view range



In [14]:
# Setting 1s Bins
bins = np.linspace(0,1200,1201) # np.linspace(start, end, steps in between)
print(bins)


print("done step 3")

[0.000e+00 1.000e+00 2.000e+00 ... 1.198e+03 1.199e+03 1.200e+03]
done step 3


# 4. Construct filelist from folder path (No input required, just run)

In [15]:
os.chdir(folder_path) # setting your working directory so that your images will be saved here

filelist = list() # empty list
for root, dirs, files in os.walk(folder_path): # this for loop goes through your folder 
    for name in files:
        if name.endswith('.dat'): # and takes out all files with a .dat (file that contains your data)
            if "_" in name.split(".")[-2]:
                filepath = os.path.join(root, name) # Notes down the file path of each data file
                filelist.append(filepath) # saves it into the list

if not filelist:
    raise FileNotFoundError("No .dat files found in the selected folder!")
else:
    print(f"Number of .dat files to process: {len(filelist)}")
    # print(f"Example of first and last file saved: {filelist[0]}, {filelist[-1]}") 

print('done step 4')

Number of .dat files to process: 317
done step 4


In [18]:
StrainNames

{1: 'N2',
 2: 'AMshABLATE_nsIs109',
 3: 'ced-10_n3246',
 4: 'ced-5_n2002',
 5: 'delm-1_ok1226',
 6: 'delm-2_ok1822',
 7: 'dyf-7_m537',
 8: 'gcy-18_nj38',
 9: 'ifa-4_ok1717',
 10: 'ifa-4_ok1734',
 11: 'lgc-55_n4331',
 12: 'lit-1_ne1991',
 13: 'mgl-2_tm355',
 14: 'mgl-2xmgl-1_tm355xtm1811',
 15: 'mir-235_n4504',
 16: 'pde-1_ok2924',
 17: 'ptp-3_ok2428',
 18: 'syg-1_ok3640',
 19: 'tax-2_ok3403',
 20: 'tax-2_p671',
 21: 'tax-4_ks11',
 22: 'tax-4_p678',
 23: 'unc-6_e78',
 24: 'ver-1_ok1738',
 25: 'ztf-16_ok1916'}

In [22]:
for strain in tqdm(StrainNames.values()): 
    if not strain == '':
        strain_filelist = [x for x in filelist if strain in x] # Goes through the list and filters for keyword
        Strain_N = len(strain_filelist) # Finds the number of plates per strain
        if Strain_N == 0:
            raise AssertionError ('{} is not a good identifier'.format(strain))
        else:
            pass
            print(f'Strain {strain}')
            print(f'Number of plates: {Strain_N}') 
            
            # visiting files in this strain
            strain_filelist = [file for file in filelist if strain in file]
            df_list=[]
            for i, file in enumerate(strain_filelist):
                print(file)
                # if file.split('/')[-1].startswith('._'):
                #     pass
                # else:
                #     try:
                #         print(f"File: {file}")
                #         df= pd.read_csv(file, sep=' ', header = None, encoding_errors='ignore')
                #         df['Plate_id'] = file.split('/')[-1].split('_')[-1].split('.')[0]

  0%|          | 0/25 [00:00<?, ?it/s]

Strain N2
Number of plates: 75
/Volumes/RankinLabMehak_SSD/Glia_Genes_Screen_2025/mgl-2_tm355/20241024_171133/N2_5x4_f96h20C_600s30x10s10s_B1024.dat
/Volumes/RankinLabMehak_SSD/Glia_Genes_Screen_2025/N2/20240724_025822/N2_5x4_f96h20C_600s30x10s10s_A0724.dat
/Volumes/RankinLabMehak_SSD/Glia_Genes_Screen_2025/N2/20240724_035049/N2_5x4_f96h20C_600s30x10s10s_A0724.dat
/Volumes/RankinLabMehak_SSD/Glia_Genes_Screen_2025/N2/20240724_094826/N2_5x4_f96h20C_600s31x10s10s_B0724.dat
/Volumes/RankinLabMehak_SSD/Glia_Genes_Screen_2025/N2/20240724_095505/N2_5x4_f96h20C_600s31x10s10s_C0724.dat
/Volumes/RankinLabMehak_SSD/Glia_Genes_Screen_2025/N2/20240724_103519/N2_5x4_f96h20C_600s31x10s10s_B0724.dat
/Volumes/RankinLabMehak_SSD/Glia_Genes_Screen_2025/N2/20240724_104235/N2_5x4_f96h20C_600s31x10s10s_C0724.dat
/Volumes/RankinLabMehak_SSD/Glia_Genes_Screen_2025/N2/20240727_144831/N2_5x4_f96h20C_600s31x10s10s_B0727.dat
/Volumes/RankinLabMehak_SSD/Glia_Genes_Screen_2025/N2/20240731_053747/N2_5x4_f96h20C_600

In [32]:
# str.join(file.split('/')[-2:])
file.split('/')[-2]+"_"+ file.split('/')[-1].split('_')[-1].split('.')[0]

'20250313_220015_C0313'

In [33]:
file.split('/')[-1].split('_')[-1].split('.')[0]

'C0313'

In [28]:
file.split('/')[-1].split('_')[-1].split('.')[0]

'C0313'

# 5. Process Data Function (No input required, just run)

In [34]:
def ProcessData(strain, experiment_counter): 
    """
    Filters and processes .dat files matching the given strain.

    Parameters: 
        strain (str): keyword to match in the files

    Returns:
        dict: N (Plate number) and Dataframe with required columns 
              ("time", "dura", "dist", "prob", "speed", "plate", "Date",
              "Plate_id", "Screen")

    """
    strain_filelist = [x for x in filelist if strain in x] # Goes through the list and filters for keyword
    Strain_N = len(strain_filelist) # Finds the number of plates per strain
    if Strain_N == 0:
        raise AssertionError ('{} is not a good identifier'.format(strain))
    else:
        pass
        print(f'Strain {strain}')
        print(f'Number of plates: {Strain_N}') 
        
        # visiting files in this strain
        strain_filelist = [file for file in filelist if strain in file]
        df_list=[]
        for i, file in enumerate(strain_filelist):
            if file.split('/')[-1].startswith('._'):
                pass
            else:
                try:
                    print(f"File: {file}")
                    df= pd.read_csv(file, sep=' ', header = None, encoding_errors='ignore')
                    df['Plate_id'] = file.split('/')[-2]+"_"+ file.split('/')[-1].split('_')[-1].split('.')[0]
                    df['Date'] = file.split('/')[-2].split('_')[0]
                    df['Screen'] = file.split('/')[-4]
                    df['Experiment'] = experiment_counter
                    experiment_counter = 1+experiment_counter
                    df_list.append(df)
                except:
                    print(f"error in file {file}")
                    pass
        DF_Total = pd.concat(df_list, ignore_index = True)
        DF_Total = DF_Total.rename( 
                    {0:'Time',
                    1:'n',
                    2:'Number',
                    3:'Instantaneous Speed',
                    4:'Interval Speed',
                    5:'Bias',
                    6:'Tap',
                    7:'Puff',
                    8:'x',
                    9:'y',
                    10:'Morphwidth',
                    11:'Midline',
                    12:'Area',
                    13:'Angular Speed',
                    14:'Aspect Ratio',
                    15:'Kink',
                    16:'Curve',
                    17:'Crab',
                    18:'Pathlength'}, axis=1)
        
        # check function here for NaN Columns
        DF_Total['plate'] = 0

        print("---------------------------------------------------------------------------------------------------------------------------------------------------------------------------")

    return{
            'N': Strain_N,
            'Confirm':DF_Total,
            'experiment_counter': experiment_counter
            # 'Final': DF_Final
    }


print('done step 5')

done step 5


# 6.1 Process Data

Create a dictionary `StrainNames` that contains all the genotype/strain names from each file path

In [35]:
genotype=[]
for f in filelist:
    genotype.append(f.split('/')[-3])

genotypes=np.unique(genotype).tolist()

if Screen =="Neuron_Genes_Screen":
    genotypes.insert(0, genotypes.pop(genotypes.index("N2_XJ1")))
    genotypes.insert(0, genotypes.pop(genotypes.index("N2_N2")))
else:
    genotypes.insert(0, genotypes.pop(genotypes.index("N2")))

nstrains = list(range(1, len(genotypes) + 1))
StrainNames = {nstrains[i]: genotypes[i] for i in range(len(nstrains))}

print(f"Number of genotypes/strains in the experiment: {len(genotypes)}")

# Display the first 5 Strain names in the experiment
for k in list(StrainNames)[:5]:
    print(f"{k}: {StrainNames[k]}")


print("done step 6.1")

# <---------------- Test element to use for dictionary buidling -------------------
# s = '/Users/Joseph/Desktop/OnFoodOffFoodTest/N2_OnFood/20220401_163048/N2_10x1_n96h20C_360sA0401_ka.00065.dat'
# slist=s.split('/')[5]
# print(slist)
# print(list(range(1,5+1)))

Number of genotypes/strains in the experiment: 25
1: N2
2: AMshABLATE_nsIs109
3: ced-10_n3246
4: ced-5_n2002
5: delm-1_ok1226
done step 6.1


# 6.2 Process Data (just run this cell)

Pass each strain through `ProcessData()` function 

In [36]:
DataLists = [0] # generates empty list at index 0 because we want indexing to start at 1 
                # when I say #1, I want the first point, not the second point

experiment_counter = 1

# the loop below goes through the dictionary in step 6.1 and processes data
# and appends all data into a list of dataframes
for s in tqdm(StrainNames.values()): 
    if not s == '':
        result = ProcessData(s, experiment_counter)
        DataLists.append(result['Confirm'])
        experiment_counter = result['experiment_counter'] 

print('done step 6.2')

  0%|          | 0/25 [00:00<?, ?it/s]

Strain N2
Number of plates: 75
File: /Volumes/RankinLabMehak_SSD/Glia_Genes_Screen_2025/mgl-2_tm355/20241024_171133/N2_5x4_f96h20C_600s30x10s10s_B1024.dat
File: /Volumes/RankinLabMehak_SSD/Glia_Genes_Screen_2025/N2/20240724_025822/N2_5x4_f96h20C_600s30x10s10s_A0724.dat
File: /Volumes/RankinLabMehak_SSD/Glia_Genes_Screen_2025/N2/20240724_035049/N2_5x4_f96h20C_600s30x10s10s_A0724.dat
File: /Volumes/RankinLabMehak_SSD/Glia_Genes_Screen_2025/N2/20240724_094826/N2_5x4_f96h20C_600s31x10s10s_B0724.dat
File: /Volumes/RankinLabMehak_SSD/Glia_Genes_Screen_2025/N2/20240724_095505/N2_5x4_f96h20C_600s31x10s10s_C0724.dat
File: /Volumes/RankinLabMehak_SSD/Glia_Genes_Screen_2025/N2/20240724_103519/N2_5x4_f96h20C_600s31x10s10s_B0724.dat
File: /Volumes/RankinLabMehak_SSD/Glia_Genes_Screen_2025/N2/20240724_104235/N2_5x4_f96h20C_600s31x10s10s_C0724.dat
File: /Volumes/RankinLabMehak_SSD/Glia_Genes_Screen_2025/N2/20240727_144831/N2_5x4_f96h20C_600s31x10s10s_B0727.dat
File: /Volumes/RankinLabMehak_SSD/Glia_G

# Convert float64 data to float32 to reduce memory load (can also convert to 16 if needed)

For plain english:

float16 = 4 decimal points

float32 = 8 decimal points

float64 = 16 decimal points

more decimal points = more data/memory that computer has to keep track of

In [38]:
# commented out this section in case memory load needs to be reduced

for n in tqdm(DataLists[1:]):
    print(n)
    TestData=n
    TestData[TestData.select_dtypes(np.float64).columns]=TestData.select_dtypes(np.float64).astype(np.float16)
    

  0%|          | 0/25 [00:00<?, ?it/s]

                Time   n  Number  Instantaneous Speed  Interval Speed  \
0           0.007000   0       0             0.000000        0.000000   
1           0.052002   0       0             0.000000        0.000000   
2           0.098022   0       0             0.000000        0.000000   
3           0.137939   0       0             0.000000        0.000000   
4           0.176025   0       0             0.000000        0.000000   
...              ...  ..     ...                  ...             ...   
1638676  1200.000000  76      28             0.101624        0.085205   
1638677  1200.000000  76      28             0.118408        0.100525   
1638678  1200.000000  75      28             0.086487        0.072571   
1638679  1200.000000  75      28             0.000000        0.000000   
1638680  1200.000000  76      28             0.000000        0.000000   

            Bias  Tap  Puff        x     y  Morphwidth   Midline      Area  \
0        0.00000    0     0   0.0000   0.0   

/Users/gurmehak/miniforge3/envs/rankinlab/lib/python3.11/site-packages/pandas/io/formats/format.py:1458: RuntimeWarning: overflow encountered in cast
  has_large_values = (abs_vals > 1e6).any()
/Users/gurmehak/miniforge3/envs/rankinlab/lib/python3.11/site-packages/pandas/io/formats/format.py:1458: RuntimeWarning: overflow encountered in cast
  has_large_values = (abs_vals > 1e6).any()
/Users/gurmehak/miniforge3/envs/rankinlab/lib/python3.11/site-packages/pandas/io/formats/format.py:1458: RuntimeWarning: overflow encountered in cast
  has_large_values = (abs_vals > 1e6).any()
/Users/gurmehak/miniforge3/envs/rankinlab/lib/python3.11/site-packages/pandas/io/formats/format.py:1458: RuntimeWarning: overflow encountered in cast
  has_large_values = (abs_vals > 1e6).any()
/Users/gurmehak/miniforge3/envs/rankinlab/lib/python3.11/site-packages/pandas/io/formats/format.py:1458: RuntimeWarning: overflow encountered in cast
  has_large_values = (abs_vals > 1e6).any()
/Users/gurmehak/miniforge3/env

# 7. Grouping Data and Naming

This step takes all the individual strain data (processed in Step 6) and combines them into single dataframe, filters for time window 490s - 590s, drops unwanted columns. 

The final processed data `Baseline_data` is ready for analysis.

In [ ]:
base=pd.concat(df.assign(dataset=StrainNames.get(i+1)) for i, df in enumerate(DataLists[1:]))

base[['Gene', 'Allele']] = base['dataset'].str.split(pat='_', n=1, expand=True)

base['Screen']=Screen

base['Allele'] = base['Allele'].fillna('N2')

Baseline_data = Baseline_data.drop(columns=["Tap", "Puff", "x","y", "Experiment"]).dropna().reset_index(drop=True)



Baseline_data = Baseline_data[((Baseline_data.Time<=590)&(Baseline_data.Time >=490))] 

Baseline_data.head()

/Users/gurmehak/miniforge3/envs/rankinlab/lib/python3.11/site-packages/pandas/io/formats/format.py:1458: RuntimeWarning: overflow encountered in cast
  has_large_values = (abs_vals > 1e6).any()


,Time,n,Number,Instantaneous Speed,Interval Speed,Bias,Morphwidth,Midline,Area,Angular Speed,Aspect Ratio,Kink,Curve,Crab,Pathlength,Plate_id,Date,Screen,plate,dataset,Gene,Allele
12193,490.0,28,19,0.046997,0.047913,0.222046,0.091919,1.127930,0.130005,3.000000,0.244019,43.68750,30.093750,0.005600,6.722656,A1025,20241025,ASD_WGS_Screen,0,N2,N2,N2
12194,490.0,28,19,0.052795,0.048187,0.166992,0.091980,1.136719,0.130005,2.900391,0.243042,43.09375,30.906250,0.005600,6.726562,A1025,20241025,ASD_WGS_Screen,0,N2,N2,N2
12195,490.0,28,19,0.057587,0.059113,0.222046,0.091919,1.133789,0.130371,3.400391,0.250000,44.90625,31.093750,0.005699,6.726562,A1025,20241025,ASD_WGS_Screen,0,N2,N2,N2
12196,490.0,28,19,0.056793,0.064392,0.166992,0.092224,1.132812,0.130737,3.500000,0.246948,46.09375,30.296875,0.007702,6.726562,A1025,20241025,ASD_WGS_Screen,0,N2,N2,N2
12197,490.0,28,19,0.058899,0.064697,0.111023,0.092102,1.138672,0.130859,3.800781,0.250000,46.68750,30.203125,0.007301,6.730469,A1025,20241025,ASD_WGS_Screen,0,N2,N2,N2


In [43]:
Baseline_data.shape

(1087471, 22)

## Creating Post Stimulus Data 

In [44]:
# similar filters as baseline data

Post_stimulus_data_pre=pd.concat(df.assign(dataset=StrainNames.get(i+1)) for i,df in enumerate(DataLists[1:]))

Post_stimulus_data_pre[['Gene', 'Allele']] = Post_stimulus_data_pre['dataset'].str.split('_', n=1, expand=True)

Post_stimulus_data_pre['Allele'] = Post_stimulus_data_pre['Allele'].fillna('N2')

Post_stimulus_data_pre = Post_stimulus_data_pre.drop(columns=["Puff", "x","y"]).dropna().reset_index(drop=True)

Post_stimulus_data_pre['Screen']=Screen

Post_stimulus_data_pre = Post_stimulus_data_pre[((Post_stimulus_data_pre.Time>599))]

Post_stimulus_data_pre['Time'] = round(Post_stimulus_data_pre['Time']).astype('int')

In [45]:
# Add continuous tap numbers from 1 to 31 for each experiment
# E.g., Experiment 1 has taps 1-31, Experiment 2 has taps 1-31 and so on..

Post_stimulus_data_pre['Tap_num'] = Post_stimulus_data_pre.groupby(['Experiment'])['Tap'].cumsum()

Post_stimulus_data_pre.head()

/Users/gurmehak/miniforge3/envs/rankinlab/lib/python3.11/site-packages/pandas/io/formats/format.py:1458: RuntimeWarning: overflow encountered in cast
  has_large_values = (abs_vals > 1e6).any()


,Time,n,Number,Instantaneous Speed,Interval Speed,Bias,Tap,Morphwidth,Midline,Area,Angular Speed,Aspect Ratio,Kink,Curve,Crab,Pathlength,Plate_id,Date,Screen,Experiment,plate,dataset,Gene,Allele,Tap_num
14910,600,25,16,0.081116,0.068726,-0.125,0,0.090271,1.115234,0.126099,3.699219,0.268066,42.00000,29.703125,0.005699,4.867188,A1025,20241025,ASD_WGS_Screen,1,0,N2,N2,N2,0
14911,600,25,16,0.078308,0.068726,0.000,0,0.091125,1.122070,0.125854,4.199219,0.263916,43.81250,30.500000,0.003500,4.867188,A1025,20241025,ASD_WGS_Screen,1,0,N2,N2,N2,0
14912,600,25,16,0.071106,0.064514,0.000,0,0.090027,1.120117,0.125488,4.699219,0.268066,45.31250,30.500000,0.003700,4.867188,A1025,20241025,ASD_WGS_Screen,1,0,N2,N2,N2,0
14913,600,25,16,0.059998,0.048706,0.000,0,0.090210,1.118164,0.125488,3.900391,0.269043,45.68750,30.500000,0.005299,4.867188,A1025,20241025,ASD_WGS_Screen,1,0,N2,N2,N2,0
14914,600,25,16,0.055115,0.041992,-0.125,0,0.090820,1.116211,0.126099,3.699219,0.270020,45.09375,30.406250,0.005501,4.867188,A1025,20241025,ASD_WGS_Screen,1,0,N2,N2,N2,0


In [46]:
# Create windows from 7s to 9.5s post a tap ("Tap"=1) for each experiment
# and concatenate all these wondows into a single dataframe

Post_stimulus_data = []

for exp in Post_stimulus_data_pre['Experiment'].unique(): # loop through each experiment separately 
    df = Post_stimulus_data_pre[Post_stimulus_data_pre['Experiment'] == exp]  
    tap_times = df[df['Tap'] == 1]['Time']  # get times where tap occured

    for t in tap_times: 
        window = df[(df['Time'] >= t + 7) & (df['Time'] <= t + 9.5)]
        Post_stimulus_data.append(window)

Post_stimulus_data = pd.concat(Post_stimulus_data)

Post_stimulus_data.head()


/Users/gurmehak/miniforge3/envs/rankinlab/lib/python3.11/site-packages/pandas/io/formats/format.py:1458: RuntimeWarning: overflow encountered in cast
  has_large_values = (abs_vals > 1e6).any()


,Time,n,Number,Instantaneous Speed,Interval Speed,Bias,Tap,Morphwidth,Midline,Area,Angular Speed,Aspect Ratio,Kink,Curve,Crab,Pathlength,Plate_id,Date,Screen,Experiment,plate,dataset,Gene,Allele,Tap_num
15092,607,27,18,0.070190,0.060608,-0.111023,0,0.091614,1.101562,0.125732,4.800781,0.451904,72.1250,43.00000,0.015404,4.136719,A1025,20241025,ASD_WGS_Screen,1,0,N2,N2,N2,1
15093,607,27,18,0.080627,0.073425,-0.111023,0,0.091492,1.099609,0.125732,5.199219,0.460938,71.8125,42.31250,0.020096,4.136719,A1025,20241025,ASD_WGS_Screen,1,0,N2,N2,N2,1
15094,607,28,18,0.080627,0.073792,-0.111023,0,0.091309,1.113281,0.126831,6.000000,0.467041,70.6875,43.90625,0.019501,4.136719,A1025,20241025,ASD_WGS_Screen,1,0,N2,N2,N2,1
15095,607,27,18,0.069580,0.066772,-0.222046,0,0.092285,1.105469,0.126099,6.199219,0.455078,71.0000,43.40625,0.017197,4.132812,A1025,20241025,ASD_WGS_Screen,1,0,N2,N2,N2,1
15096,607,27,18,0.076721,0.068726,-0.222046,0,0.091919,1.103516,0.125610,6.601562,0.458008,71.6250,43.09375,0.018707,4.132812,A1025,20241025,ASD_WGS_Screen,1,0,N2,N2,N2,1


In [47]:
# Aggregate columns by "Experiment" + "Tap_num" by taking their means

Post_stimulus_data = Post_stimulus_data.groupby(['Experiment', 'Tap_num','Screen','Date','Plate_id','Gene','Allele','dataset', "plate"]).agg({
    'Time': 'min', # take minimum valu of time instead of mean
    'n': 'mean',
    'Number': 'mean',
    'Instantaneous Speed': 'mean',
    'Interval Speed' : 'mean',
    'Bias': 'mean',
    'Tap': 'mean',
    'Morphwidth': 'mean',
    'Midline': 'mean',
    'Area': 'mean',
    'Angular Speed': 'mean',
    'Aspect Ratio': 'mean',
    'Kink': 'mean',
    'Curve': 'mean',
    'Crab': 'mean',
    'Pathlength': 'mean'
})

Post_stimulus_data = Post_stimulus_data.reset_index()

Post_stimulus_data

,Experiment,Tap_num,Screen,Date,Plate_id,Gene,Allele,dataset,plate,Time,n,Number,Instantaneous Speed,Interval Speed,Bias,Tap,Morphwidth,Midline,Area,Angular Speed,Aspect Ratio,Kink,Curve,Crab,Pathlength
0,1,1,ASD_WGS_Screen,20241025,A1025,N2,N2,N2,0,607,27.096774,18.000000,0.087289,0.090278,-0.152349,0.0,0.092326,1.110147,0.127160,7.942163,0.425384,66.743446,41.800907,0.016138,4.108871
1,1,2,ASD_WGS_Screen,20241025,A1025,N2,N2,N2,0,617,28.016667,20.533333,0.146644,0.131465,0.297072,0.0,0.089995,1.123535,0.126896,11.654948,0.375224,68.082291,39.698959,0.023494,3.702246
2,1,3,ASD_WGS_Screen,20241025,A1025,N2,N2,N2,0,627,29.000000,23.555556,0.194838,0.126656,0.596924,0.0,0.093820,1.129681,0.129061,10.315910,0.319154,52.993553,35.892857,0.024813,2.806517
3,1,4,ASD_WGS_Screen,20241025,A1025,N2,N2,N2,0,637,27.145455,25.000000,0.235256,0.104865,0.933754,0.0,0.093574,1.145224,0.130513,10.019815,0.292494,46.286930,34.378410,0.029297,3.431854
4,1,5,ASD_WGS_Screen,20241025,A1025,N2,N2,N2,0,647,25.166667,19.458333,0.207614,0.096355,0.939473,0.0,0.091526,1.142965,0.131775,7.616943,0.274516,43.262371,35.502605,0.021729,4.502686
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15672,510,27,ASD_WGS_Screen,20250302,B0302,wdr-20,tm12622,wdr-20_tm12622,0,867,28.081081,22.000000,0.187711,0.101990,0.866251,0.0,0.098484,0.940905,0.112357,7.164802,0.217810,33.164272,26.283783,0.012532,9.261402
15673,510,28,ASD_WGS_Screen,20250302,B0302,wdr-20,tm12622,wdr-20_tm12622,0,877,29.791667,22.687500,0.190353,0.092055,0.930196,0.0,0.097028,0.946045,0.113825,7.843994,0.244143,35.598957,27.779623,0.013594,10.010742
15674,510,29,ASD_WGS_Screen,20250302,B0302,wdr-20,tm12622,wdr-20_tm12622,0,887,30.000000,24.532258,0.178488,0.101608,0.816501,0.0,0.099646,0.946636,0.115591,7.065965,0.226368,35.795868,26.098791,0.013418,9.490802
15675,510,30,ASD_WGS_Screen,20250302,B0302,wdr-20,tm12622,wdr-20_tm12622,0,897,31.000000,26.000000,0.167146,0.087110,0.901973,0.0,0.099261,0.944824,0.115200,7.384563,0.245031,41.005386,27.784752,0.012508,10.181708


In [50]:
Post_stimulus_data.plate.unique()

array([0])

In [49]:
print('done step 7')

done step 7


# Save dataframe as `.csv`

In [51]:
Baseline_data.to_csv(f"{Screen}_baseline_output.csv")
print('saved Baseline data as .csv!')

saved Baseline data as .csv!


In [52]:
Post_stimulus_data.to_csv(f"{Screen}_post_stimulus.csv")
print('saved Post stimulus data as .csv!')

saved Post stimulus data as .csv!


# Done!